In [1]:
import cv2
import numpy as np
import os
import torch
from model import EncoderCNN, DecoderRNN
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

version 2


In [ ]:
embed_size = 256#<-
hidden_size = 512#<-
vocab_size = 9955
encoder_file = 'v3encoder-4.pkl' 
decoder_file = 'v3decoder-4.pkl'

In [ ]:
path=input()

In [ ]:
encoder = EncoderCNN(embed_size)
encoder.eval()
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)
decoder.eval()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load the trained weights.
encoder.load_state_dict(torch.load(os.path.join('', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('', decoder_file)))
# Move models to GPU if CUDA is available.
encoder.to(device)
print(encoder.to(device))
decoder.to(device)
print(decoder.to(device))

In [ ]:
dicctionary= pickle.load( open( "dicctionary.pkl", "rb" ) )

In [ ]:
def clean_sentence(output,dicttionary):
    sentence=[]
    for i in output:
        sentence.append(dicctionary[i])
    indices = [i for i, s in enumerate(sentence) if '<end>' in s]
    sentence=sentence[1:indices[0]]
    sentence=' '.join(sentence)
    return sentence

In [ ]:
for file in np.random.choice(os.listdir(path), 1,replace=False):

    path_to_file=path+file
    imgoriginal=cv2.imread(path_to_file)
    im=imgoriginal.copy()
    im=im/255
    im=torch.tensor(im.transpose(2, 0, 1),dtype=torch.float32)
    encoder.eval()
    with torch.no_grad():
        image = im.to(device)
        # Obtain the embedded image features.
        features = encoder(image.unsqueeze(0)).unsqueeze(1)
        # Pass the embedded image features through the model to get a predicted caption.
    output = decoder.sample(features)
    sentence = clean_sentence(output,dicctionary)
    print('example sentence:', sentence)
    plt.imshow(cv2.cvtColor(imgoriginal, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
path_to_file